In [ ]:
#| default_exp constraintscan

In [ ]:
#| export
import numpy as np
import pandas as pd
from mmon_gcm.solving import pfba_Weighted
from mmon_gcm.supermodel import SuperModel

In [ ]:
#| export

def normalise_within_bounds(constraint_series, parameter_df):
    lower_bound = parameter_df["Lower"]
    upper_bound = parameter_df["Upper"]

    return (constraint_series * (upper_bound - lower_bound) + lower_bound)

In [ ]:
#| export

def generate_constraints_df(constraints_df, params_df):
    '''
    This functions takes a list of constraints along with a dataframe that contains the bounds for the parameters that these constraints should apply to
    It uses the bounds from the dataframe with the values from the constraints_list to generate a dataframe of constraints for running a constraint scan
    '''

    # firstly, use the values between 0-1 to get a value for the parameter between the bounds
    values_df = constraints_df.apply(normalise_within_bounds, args=([params_df]), axis=1)

    # next, remove solutions where the open aperture is larger than the closed

    open_less_closed_number = np.array(values_df["A_open"] < values_df["A_closed"]).sum()

    print(f'Removing {open_less_closed_number} combinations where open aperture is smaller than closed')

    values_df = values_df[values_df["A_closed"] < values_df["A_open"]]

    return values_df

In [ ]:
#| export

def solve_model_for_constraint_scan(constraints, fba_model, light_colour, printouts=False):
    
    solution_number = constraints.name

    super_model = SuperModel(constraints.T, fba_model=fba_model)
    super_model.constrain_osmolarity(printouts=printouts)
    super_model.constrain_photons(150, printouts=printouts)
    gc_atpase_upper_bound = super_model.get_atpase_constraint_value(constraints.loc["ATPase"])

    for p in [1, 2, 3, 4]:
        super_model.fba_model.reactions.get_by_id(f"PROTON_ATPase_c_gc_{p}").bounds = (0, gc_atpase_upper_bound)

    if printouts is True:
        print(f"ATPase constrained to {gc_atpase_upper_bound}")

    if light_colour == "blue":
        super_model.fba_model.reactions.Photon_tx_gc_2.upper_bound = 0
        super_model.fba_model.reactions.Photon_tx_me_2.upper_bound = 0
        
    elif light_colour == "none":
        for i in [1,2,3,4]:
            super_model.fba_model.reactions.get_by_id(f"Photon_tx_gc_{i}").upper_bound = 0

    elif light_colour == "white":
        pass

    else:
        raise ValueError("Light should be either blue or white")
        
    try:
        print(f"Solving model {solution_number}")
        super_model.fba_model, solution = pfba_Weighted(super_model.fba_model, objective="Phloem_tx_overall")
        print(f"Solved model {solution_number}")
    
    except:
        print(f"{solution_number} is infeasible")
        solution = pd.NA

    return solution